In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_article = pd.read_csv('./data/zhuanlan_article.csv',header=0)
df_article.head()

,zhuanlan_url,author,title,article_url,publishedTime,commentsCount,likesCount,content
0,https://zhuanlan.zhihu.com/c_35601954,何钦尧,贝叶斯统计观点下的拉普拉斯平滑,https://zhuanlan.zhihu.com/p/24291822,2016-12-31T17:04:53+08:00,33.0,175.0,这终于是本专栏的第一篇机器学习相关的文章，憋了几个月了，充分表现了自己肚子里实在是没什么墨水...
1,https://zhuanlan.zhihu.com/c_35601954,何钦尧,开篇,https://zhuanlan.zhihu.com/p/21703690,2016-07-24T16:43:17+08:00,21.0,66.0,其实这个专栏已经开通了好久了。然而一直没有时间写什么东西（其实现在大概也没有时间写）。想了一...
2,https://zhuanlan.zhihu.com/deeplearning-surfing,清凇,用深度学习（CNN RNN Attention）解决大规模文本分类问题 - 综述和实践,https://zhuanlan.zhihu.com/p/25928551,2017-03-26T17:52:14+08:00,86.0,757.0,近来在同时做一个应用深度学习解决淘宝商品的类目预测问题的项目，恰好硕士毕业时论文题目便是文本...
3,https://zhuanlan.zhihu.com/deeplearning-surfing,清凇,用深度学习（DNN）构建推荐系统 - Deep Neural Networks for Yo...,https://zhuanlan.zhihu.com/p/25343518,2017-02-22T20:31:00+08:00,37.0,247.0,这篇论文 Deep Neural Networks for YouTube Recommen...
4,https://zhuanlan.zhihu.com/deeplearning-surfing,清凇,深度强化学习（Deep Reinforcement Learning）入门：RL base ...,https://zhuanlan.zhihu.com/p/25239682,2017-02-15T22:40:33+08:00,31.0,233.0,过去的一段时间在深度强化学习领域投入了不少精力，工作中也在应用DRL解决业务问题。子曰：温故...


In [5]:
df_info = pd.read_csv('./data/zhuanlan_info.csv',header=None)
df_info.head()

,0,1,2,3,4,5
0,机器学习笔记,子楠,机器学习的笔记。非常浅显。不得不说ng的课很适合入门，几乎不需要太多的数学基础,https://zhuanlan.zhihu.com/mlearn,22,5873
1,无痛的机器学习,冯超,专栏主营业务：让更多人能看的懂的机器学习科普+进阶文章。欢迎各位大神投稿或协助审阅,https://zhuanlan.zhihu.com/hsmyy,76,15147
2,Python 与 机器学习,射命丸咲,NaN,https://zhuanlan.zhihu.com/carefree0910-pyml,57,8687
3,开始学习机器人,Top Liu,NaN,https://zhuanlan.zhihu.com/learn-robotics,42,4744
4,机器学习与数据挖掘,Eric Leo,NaN,https://zhuanlan.zhihu.com/Industry4,3,175


In [13]:
df_info.columns = ['zhuanlan_name','zhuanlan_author','description','zhuanlan_url','articles_count','followers']

In [7]:
df_info['articles_count'] = df_info['articles_count'].astype('int')
df_info['followers'] = df_info['followers'].astype('int')

df_info = df_info[df_info['articles_count']>0]

# 评论数、点赞数统计

In [14]:
df_data = pd.merge(df_info,df_article,how='left',on='zhuanlan_url')

In [15]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6585 entries, 0 to 6584
Data columns (total 13 columns):
zhuanlan_name      6585 non-null object
zhuanlan_author    6585 non-null object
description        5320 non-null object
zhuanlan_url       6585 non-null object
articles_count     6585 non-null int64
followers          6585 non-null int64
author             6585 non-null object
title              6584 non-null object
article_url        6585 non-null object
publishedTime      6585 non-null object
commentsCount      6585 non-null float64
likesCount         6585 non-null float64
content            6543 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 720.2+ KB


In [17]:
stat_feat = ['min','mean','max']
comLike_df = df_data.groupby('zhuanlan_name')['commentsCount','likesCount'].agg(stat_feat).reset_index()
# statistic_df.columns = ['uid'] + ['loan_' + col for col in stat_feat]

In [19]:
comLike_df.columns = ['zhuanlan_name','commentsCount_min','commentsCount_mean','commentsCount_max',\
                     'likesCount_min','likesCount_mean','likesCount_max']

In [23]:
comLike_df = comLike_df.sort_values(by=['commentsCount_mean','likesCount_mean'],ascending=False)

In [26]:
comLike_df = pd.merge(comLike_df,df_info[['zhuanlan_name','articles_count','followers']],how='left',on='zhuanlan_name')

In [28]:
comLike_df['articles_count_log'] = comLike_df['articles_count'].map(lambda x: np.log(x+1))
comLike_df['followers_log'] = comLike_df['followers'].map(lambda x: np.log(x+1))

In [29]:
comLike_df.head()

,zhuanlan_name,commentsCount_min,commentsCount_mean,commentsCount_max,likesCount_min,likesCount_mean,likesCount_max,articles_count,followers,articles_count_log,followers_log
0,PMCAFF产品社区,0.0,120.500000,3729.0,1.0,68.315789,766.0,38,3767,3.663562,8.234300
1,船长的机器学习海志,94.0,94.000000,94.0,355.0,355.000000,355.0,1,94,0.693147,4.553877
2,什么值得爬,6.0,87.000000,133.0,79.0,474.333333,754.0,3,3242,1.386294,8.084254
3,挖掘知乎里有趣的东西,6.0,72.678571,167.0,46.0,711.607143,1953.0,28,36321,3.367296,10.500179
4,AI带路党,12.0,71.000000,226.0,217.0,881.833333,2006.0,6,4451,1.945910,8.401109


# 计算专栏推荐指数

In [30]:
comLike_df.loc[:,'zhuanlan_REI'] = (comLike_df['commentsCount_mean'] + comLike_df['likesCount_mean']*0.5)*\
                                    comLike_df['articles_count_log']*comLike_df['followers_log']

In [31]:
comLike_df = comLike_df.sort_values(by=['zhuanlan_REI'],ascending=False)
comLike_df.head(10)

,zhuanlan_name,commentsCount_min,commentsCount_mean,commentsCount_max,likesCount_min,likesCount_mean,likesCount_max,articles_count,followers,articles_count_log,followers_log,zhuanlan_REI
3,挖掘知乎里有趣的东西,6.0,72.678571,167.0,46.0,711.607143,1953.0,28,36321,3.367296,10.500179,15149.932513
5,AI Insight,4.0,69.285714,399.0,73.0,658.571429,4190.0,14,6004,2.708050,8.700348,9390.732832
4,AI带路党,12.0,71.000000,226.0,217.0,881.833333,2006.0,6,4451,1.945910,8.401109,8368.712669
18,智能单元,1.0,37.223404,195.0,9.0,265.978723,1655.0,94,37133,4.553877,10.522288,8156.120069
23,Hello 陈然！,1.0,32.633333,279.0,30.0,310.266667,2697.0,30,11209,3.433987,9.324562,6012.368455
48,Python中文社区,0.0,21.670782,189.0,1.0,133.761317,1561.0,243,34898,5.497168,10.460213,5091.845342
0,PMCAFF产品社区,0.0,120.500000,3729.0,1.0,68.315789,766.0,38,3767,3.663562,8.234300,4665.543728
24,程序猿数据爱好者,0.0,32.436620,425.0,1.0,178.746479,2511.0,71,3802,4.276666,8.243546,4294.393402
17,人工智能应用系列,0.0,37.250000,258.0,15.0,357.500000,2503.0,8,4069,2.197225,8.311398,3944.593851
8,硅谷老实人,3.0,43.875000,192.0,17.0,158.833333,723.0,48,3108,3.891820,8.042056,3858.811973


# 发帖周期计算

In [61]:
from datetime import datetime

period_df = df_data[['zhuanlan_name','zhuanlan_author','zhuanlan_url','articles_count','author','publishedTime']]
period_df.loc[:,'publishedTime'] = period_df.loc[:,'publishedTime'].map(lambda x : x.split('T')[0])

/Users/wangle/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## 第一次发帖时间

In [49]:
period_df = period_df.sort_values(by=['zhuanlan_name','publishedTime'])

In [44]:
first_pub = period_df.copy()
first_pub.drop_duplicates(subset='zhuanlan_name', keep='first', inplace=True)

In [46]:
first_pub = first_pub[['zhuanlan_name','publishedTime']]

## 最后一次发帖时间

In [47]:
last_pub = period_df.copy()
last_pub.drop_duplicates(subset='zhuanlan_name', keep='last', inplace=True)
last_pub = last_pub[['zhuanlan_name','publishedTime']]

## 专栏作者个数 (是否有投稿情况）

In [55]:
author_list_col = []
for zhuanlan in last_pub['zhuanlan_name']:
    author_list = period_df.loc[period_df['zhuanlan_name']==zhuanlan,'author'].unique().tolist()
    author_list = ' '.join(author_list)
    author_list_col.append(author_list)
    
last_pub['author_list'] = author_list_col

In [62]:
first_pub.columns = ['zhuanlan_name','first_publishedTime']
last_pub.columns = ['zhuanlan_name','last_publishedTime','author_list']

period_df = period_df[['zhuanlan_name','zhuanlan_url','articles_count']]
period_df.drop_duplicates(subset='zhuanlan_name', keep='last', inplace=True)

period_df = pd.merge(first_pub,period_df,how='left',on='zhuanlan_name')
period_df = pd.merge(period_df,last_pub,how='left',on='zhuanlan_name')

In [63]:
period_df.head()

,zhuanlan_name,first_publishedTime,zhuanlan_url,articles_count,last_publishedTime,author_list
0,8层会议室,2017-04-06,https://zhuanlan.zhihu.com/8thfloor,15,2017-11-27,孙玉玺 谢泽华 周泽南 董国盛 黑鱼 花花 叶知秋 常庆丰 潘达
1,AI Insight,2016-11-05,https://zhuanlan.zhihu.com/ai-insight,14,2017-11-05,何之源
2,AI+安全试路人,2017-06-02,https://zhuanlan.zhihu.com/taoanwang,1,2017-06-02,小超超
3,AIvsBI,2017-10-24,https://zhuanlan.zhihu.com/lxwcel,4,2017-11-07,AlphaYong
4,AI带路党,2016-10-08,https://zhuanlan.zhihu.com/ai-leader,6,2017-07-31,郑华滨 刘思聪 吴晓晖


## 根据第一次发文时间、最后一次发文时间和发文数量计算平均发文周期

In [64]:
period_df['publish_period'] = period_df['first_publishedTime'].map(lambda x : datetime.strptime('2018-1-11','%Y-%m-%d') - \
                                                            datetime.strptime(x, '%Y-%m-%d'))
period_df['publish_period'] = period_df['publish_period'].map(lambda x: x.days)
period_df.loc[:,'published_period'] = period_df['publish_period'] / period_df['articles_count']
del period_df['publish_period']

In [66]:
period_df.sort_values(by='published_period',).head(10)

,zhuanlan_name,first_publishedTime,zhuanlan_url,articles_count,last_publishedTime,author_list,published_period
387,车神的机器学习,2017-12-25,https://zhuanlan.zhihu.com/c_128528277,18,2018-01-10,Pig951001,0.944444
301,深度学习源码解析,2017-11-24,https://zhuanlan.zhihu.com/c_144268772,39,2018-01-11,小白,1.230769
9,BAT面试1000题,2017-11-10,https://zhuanlan.zhihu.com/c_140166199,40,2018-01-11,julyedu,1.550000
357,网易智能,2017-02-14,https://zhuanlan.zhihu.com/smartman163,196,2018-01-11,小羿,1.688776
112,一片神鸦社鼓,2017-12-18,https://zhuanlan.zhihu.com/yang3,11,2018-01-08,讳莫如深,2.181818
127,人工智能成长之路,2017-12-04,https://zhuanlan.zhihu.com/c_147015982,17,2018-01-09,weapon,2.235294
69,Python中文社区,2016-06-15,https://zhuanlan.zhihu.com/zimei,243,2018-01-04,段小草 iGuo xchaoinfo 笑虎 阿橙 夏洛之枫 苍冥 九茶 Jerry Wayn...,2.366255
85,TensorFlowNews,2017-07-11,https://zhuanlan.zhihu.com/TensorFlownews,77,2018-01-07,灰灰,2.389610
102,nlp相关论文笔记,2017-12-21,https://zhuanlan.zhihu.com/c_134593059,8,2018-01-10,TtC的WH,2.625000
65,PaperWeekly,2016-05-26,https://zhuanlan.zhihu.com/paperweekly,216,2018-01-05,张俊,2.754630


In [67]:
period_df = period_df.sort_values(by='published_period')
period_df = period_df[['zhuanlan_name','zhuanlan_url','author_list','articles_count','first_publishedTime','last_publishedTime','published_period']]

In [70]:
period_df = period_df.reset_index(drop = True)

with open('recommend_Index_publish_Period.txt','a+') as f:
    for i in period_df.index:
        f.write('<tr>\n')
        f.write('<td>'+ period_df.loc[i,'zhuanlan_name']+'</td>\n')
        f.write('<td>'+ period_df.loc[i,'zhuanlan_url']+'</td>\n')
        f.write('<td>'+ period_df.loc[i,'author_list']+'</td>\n')
        f.write('<td>'+ str(period_df.loc[i,'articles_count'])+'</td>\n')
        f.write('<td>'+ period_df.loc[i,'first_publishedTime']+'</td>\n') 
        f.write('<td>'+ period_df.loc[i,'last_publishedTime']+'</td>\n')
        f.write('<td>'+ str(period_df.loc[i,'published_period'])+'</td>\n')
        f.write('</tr>\n')
        f.write('\n')

In [71]:
comLike_df.columns

Index(['zhuanlan_name', 'commentsCount_min', 'commentsCount_mean',
       'commentsCount_max', 'likesCount_min', 'likesCount_mean',
       'likesCount_max', 'articles_count', 'followers', 'articles_count_log',
       'followers_log', 'zhuanlan_REI'],
      dtype='object')

In [72]:
del comLike_df['articles_count_log']
del comLike_df['followers_log']

In [73]:
comLike_df = pd.merge(period_df[['zhuanlan_name','zhuanlan_url','published_period']],comLike_df,how='outer',on='zhuanlan_name')
comLike_df = comLike_df.reset_index(drop = True)

In [76]:
comLike_df = comLike_df.reset_index(drop = True)
comLike_df = comLike_df.sort_values(by=['zhuanlan_REI'],ascending=False)
comLike_df.head()

,zhuanlan_name,zhuanlan_url,published_period,commentsCount_min,commentsCount_mean,commentsCount_max,likesCount_min,likesCount_mean,likesCount_max,articles_count,followers,zhuanlan_REI
0,挖掘知乎里有趣的东西,https://zhuanlan.zhihu.com/grapeot,36.321429,6.0,72.678571,167.0,46.0,711.607143,1953.0,28,36321,15149.932513
1,AI Insight,https://zhuanlan.zhihu.com/ai-insight,30.857143,4.0,69.285714,399.0,73.0,658.571429,4190.0,14,6004,9390.732832
2,AI带路党,https://zhuanlan.zhihu.com/ai-leader,76.666667,12.0,71.000000,226.0,217.0,881.833333,2006.0,6,4451,8368.712669
3,智能单元,https://zhuanlan.zhihu.com/intelligentunit,6.510638,1.0,37.223404,195.0,9.0,265.978723,1655.0,94,37133,8156.120069
4,Hello 陈然！,https://zhuanlan.zhihu.com/chenran,53.166667,1.0,32.633333,279.0,30.0,310.266667,2697.0,30,11209,6012.368455


In [77]:
comLike_df.columns

Index(['zhuanlan_name', 'zhuanlan_url', 'published_period',
       'commentsCount_min', 'commentsCount_mean', 'commentsCount_max',
       'likesCount_min', 'likesCount_mean', 'likesCount_max', 'articles_count',
       'followers', 'zhuanlan_REI'],
      dtype='object')

In [80]:
comLike_df = comLike_df[['zhuanlan_name','zhuanlan_REI', 'zhuanlan_url', 'published_period','articles_count','followers',
       'commentsCount_min', 'commentsCount_mean', 'commentsCount_max',
       'likesCount_min', 'likesCount_mean', 'likesCount_max']]

In [85]:
comLike_df.head(10)

,zhuanlan_name,zhuanlan_REI,zhuanlan_url,published_period,articles_count,followers,commentsCount_min,commentsCount_mean,commentsCount_max,likesCount_min,likesCount_mean,likesCount_max
0,挖掘知乎里有趣的东西,15149.932513,https://zhuanlan.zhihu.com/grapeot,36.321429,28,36321,6.0,72.678571,167.0,46.0,711.607143,1953.0
1,AI Insight,9390.732832,https://zhuanlan.zhihu.com/ai-insight,30.857143,14,6004,4.0,69.285714,399.0,73.0,658.571429,4190.0
2,AI带路党,8368.712669,https://zhuanlan.zhihu.com/ai-leader,76.666667,6,4451,12.0,71.000000,226.0,217.0,881.833333,2006.0
3,智能单元,8156.120069,https://zhuanlan.zhihu.com/intelligentunit,6.510638,94,37133,1.0,37.223404,195.0,9.0,265.978723,1655.0
4,Hello 陈然！,6012.368455,https://zhuanlan.zhihu.com/chenran,53.166667,30,11209,1.0,32.633333,279.0,30.0,310.266667,2697.0
5,Python中文社区,5091.845342,https://zhuanlan.zhihu.com/zimei,2.366255,243,34898,0.0,21.670782,189.0,1.0,133.761317,1561.0
6,PMCAFF产品社区,4665.543728,https://zhuanlan.zhihu.com/pmcaff,24.394737,38,3767,0.0,120.500000,3729.0,1.0,68.315789,766.0
7,程序猿数据爱好者,4294.393402,https://zhuanlan.zhihu.com/c_94187513,3.676056,71,3802,0.0,32.436620,425.0,1.0,178.746479,2511.0
8,人工智能应用系列,3944.593851,https://zhuanlan.zhihu.com/ai4application,38.000000,8,4069,0.0,37.250000,258.0,15.0,357.500000,2503.0
9,硅谷老实人,3858.811973,https://zhuanlan.zhihu.com/nullptr,6.604167,48,3108,3.0,43.875000,192.0,17.0,158.833333,723.0


In [83]:
with open('recommend_Index.txt','a+') as f:
    for i in comLike_df.index:
        f.write('<tr>\n')
        f.write('<td>'+ comLike_df.loc[i,'zhuanlan_name']+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'zhuanlan_REI'])+'</td>\n')
        f.write('<td>'+ comLike_df.loc[i,'zhuanlan_url']+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'published_period'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'articles_count'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'followers'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'commentsCount_min'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'commentsCount_mean'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'commentsCount_max'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'likesCount_min'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'likesCount_mean'])+'</td>\n')
        f.write('<td>'+ str(comLike_df.loc[i,'likesCount_max'])+'</td>\n')
        f.write('</tr>\n')
        f.write('\n')

In [84]:
comLike_df[comLike_df['zhuanlan_name'] == '每天都要机器学习哦']

,zhuanlan_name,zhuanlan_REI,zhuanlan_url,published_period,articles_count,followers,commentsCount_min,commentsCount_mean,commentsCount_max,likesCount_min,likesCount_mean,likesCount_max
102,每天都要机器学习哦,582.316004,https://zhuanlan.zhihu.com/leemoo,5.588235,17,1576,0.0,7.0,57.0,0.0,40.722222,173.0


# 前十推荐专栏文章内容分析

## 文章标题主题分类

In [107]:
df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/ai-insight','title']

4202           TensorFlow全新的数据读取方式：Dataset API入门教程
4203              如何优雅地用TensorFlow预测时间序列：TFTS库详细教程
4204                               AI可能真的要代替插画师了……
4205              可能是近期最好玩的深度学习模型：CycleGAN的原理与实验详解
4206                       TensorFlow中RNN实现的正确打开方式
4207             完全图解RNN、RNN变体、Seq2Seq、Attention机制
4208                      如何用TensorFlow生成令人惊艳的分形图案
4209    「Show and Tell」——图像标注（Image Caption）任务技术综述
4210                当人工智能遇见黑暗之魂：如何使用神经网络训练一个PVP AI
4211                     十图详解tensorflow数据读取机制（附代码）
4212    通过文字描述来生成二次元妹子！聊聊conditional GAN与txt2img模型
4213                        GAN学习指南：从原理入门到制作生成Demo
4214                   深度学习实践：使用Tensorflow实现快速风格迁移
4215                             训练集样本不平衡问题对CNN的影响
Name: title, dtype: object

In [112]:
import jieba
import re,pickle
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [130]:
jieba.load_userdict('jieba_dict.txt')
stop_word = pickle.load(open('stop_word.pkl','rb'))
def deal_data(text):
    text = text.upper()
    text = re.sub(' ','',text)
    text = jieba.cut(text)
    text = [word for word in text if word not in stop_word]
    return text

In [131]:
def make_corpus(data):
    corpus = []
    for line in data:
        text = str(line)
        text = deal_data(text)
        corpus.append(text)
    return corpus

In [132]:
corpus = make_corpus(df_article['title'])

In [133]:
from gensim import models,corpora,similarities

dictionary = corpora.Dictionary(corpus)
corpus_ = [dictionary.doc2bow(text) for text in corpus]
tfidf = models.TfidfModel(corpus_)
corpus_tiidf = tfidf[corpus_]

lsi = models.LsiModel(corpus_tiidf,id2word=dictionary, num_topics = 10)


In [148]:
lsi.print_topics(num_topics=10, num_words=20)

[(0,
  '-0.513*"笔记" + -0.331*"机器学习" + -0.326*"学习" + -0.218*"深度学习" + -0.215*"阅读" + -0.190*"课程" + -0.185*"神经网络" + -0.178*"PYTHON" + -0.168*"入门" + -0.136*"算法" + -0.119*"数据分析" + -0.108*"基础" + -0.090*"TENSORFLOW" + -0.089*"翻译" + -0.087*"论文" + -0.085*"模型" + -0.085*"讲" + -0.084*"回归" + -0.084*"林轩" + -0.084*"田"'),
 (1,
  '-0.459*"笔记" + 0.434*"PYTHON" + -0.378*"阅读" + 0.352*"数据分析" + 0.267*"入门" + 0.164*"可视化" + 0.142*"实例" + 0.129*"基础" + 0.103*"实践" + -0.097*"课程" + 0.094*"数据" + 0.085*"深度学习" + 0.084*"强化学习" + 0.078*"TENSORFLOW" + 0.076*"算法" + 0.074*"爬虫" + 0.070*"中" + -0.067*"论文" + 0.064*"机器学习" + 0.063*"模型"'),
 (2,
  '0.560*"机器学习" + -0.289*"数据分析" + -0.287*"PYTHON" + -0.283*"阅读" + -0.213*"笔记" + 0.202*"题" + 0.202*"算法" + -0.146*"学习" + -0.140*"可视化" + 0.134*"回归" + 0.125*"面试" + 0.122*"系列" + -0.117*"实例" + -0.113*"神经网络" + 0.111*"BAT" + 0.110*"1000" + 0.095*"田" + 0.095*"林轩" + 0.094*"讲" + 0.091*"强化学习"'),
 (3,
  '-0.555*"深度学习" + 0.324*"数据分析" + 0.275*"机器学习" + -0.236*"神经网络" + -0.232*"TENSORFLOW" + 0.197*"PYTHON" + -

In [136]:
test_text = '如何优雅地用TensorFlow预测时间序列：TFTS库详细教程'
test_text = deal_data(test_text)
test_bow = dictionary.doc2bow(test_text)
test_lsi = lsi[test_bow]

In [137]:
test_lsi

[(0, -0.16992401619492636),
 (1, 0.16650707540391096),
 (2, -0.043419034143707548),
 (3, -0.25418315178481449),
 (4, -0.14660814654199777),
 (5, 0.11924932464105902),
 (6, 0.20001282781761126),
 (7, 0.39891875774063684),
 (8, 0.69431035750226766),
 (9, -0.062935508077685526)]

In [142]:
sorted(test_lsi,key = lambda x:x[1],reverse = True)

[(8, 0.69431035750226766),
 (7, 0.39891875774063684),
 (6, 0.20001282781761126),
 (1, 0.16650707540391096),
 (5, 0.11924932464105902),
 (2, -0.043419034143707548),
 (9, -0.062935508077685526),
 (4, -0.14660814654199777),
 (0, -0.16992401619492636),
 (3, -0.25418315178481449)]

## 测试语句属于第8个主题的概率最大，为0.69

In [139]:
lsi.show_topic(8, topn=5)

[('TENSORFLOW', 0.64756969413399812),
 ('深度学习', -0.34771117197533408),
 ('算法', -0.31522069129457225),
 ('课程', 0.23557708685778173),
 ('翻译', 0.17976032673024467)]

TensorFlow对应TENSORFLOW；

教程对应课程

## 前十推荐专栏里的文章主题分布


In [143]:
def get_topic(lsi,text):
    test_text = deal_data(text)
    test_bow = dictionary.doc2bow(test_text)
    test_lsi = lsi[test_bow]
    sort_test_lsi = sorted(test_lsi,key = lambda x:x[1],reverse = True)
    topic = sort_test_lsi[0][0]
    return topic

In [146]:
from collections import Counter

grapeot = df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/grapeot','title']
topict_list = []
for text in grapeot:
    topict_list.append(get_topic(lsi,text))
print('grapeot 专栏文章主题分布：',Counter(topict_list))

grapeot 专栏文章主题分布： Counter({7: 9, 6: 8, 1: 4, 5: 2, 2: 2, 4: 2, 8: 1})


In [147]:
ai_insight = df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/ai-insight','title']
topict_list = []
for text in ai_insight:
    topict_list.append(get_topic(lsi,text))
print('ai_insight 专栏文章主题分布：',Counter(topict_list))

ai_insight 专栏文章主题分布： Counter({8: 6, 7: 4, 1: 1, 4: 1, 6: 1, 5: 1})


In [149]:
ai_leader = df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/ai-leader','title']
topict_list = []
for text in ai_leader:
    topict_list.append(get_topic(lsi,text))
print('ai_leader 专栏文章主题分布：',Counter(topict_list))

ai_leader 专栏文章主题分布： Counter({6: 2, 8: 2, 5: 1, 7: 1})


In [154]:
zhuanlan_list = ['intelligentunit','chenran','zimei','pmcaff','c_94187513','ai4application']
for zhuanlan in zhuanlan_list:
    zhuanlan_name = df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/'+zhuanlan ,'title']
    topict_list = []
    for text in zhuanlan_name:
        topict_list.append(get_topic(lsi,text))
    print('%s 专栏文章主题分布：'%zhuanlan,Counter(topict_list))

intelligentunit 专栏文章主题分布： Counter({9: 20, 8: 20, 1: 19, 4: 11, 6: 10, 5: 7, 7: 6, 2: 1})
chenran 专栏文章主题分布： Counter({1: 10, 7: 8, 5: 6, 8: 4, 6: 1, 4: 1})
zimei 专栏文章主题分布： Counter({1: 144, 7: 35, 6: 30, 2: 10, 8: 9, 5: 8, 9: 4, 4: 3})
pmcaff 专栏文章主题分布： Counter({7: 15, 1: 12, 6: 5, 5: 2, 8: 2, 4: 1, 9: 1})
c_94187513 专栏文章主题分布： Counter({7: 30, 1: 23, 6: 5, 5: 4, 4: 3, 2: 3, 8: 2, 9: 1})
ai4application 专栏文章主题分布： Counter({1: 4, 8: 2, 9: 1, 6: 1})


In [155]:
nullptr = df_article.loc[df_article['zhuanlan_url']=='https://zhuanlan.zhihu.com/'+'nullptr' ,'title']
topict_list = []
for text in nullptr:
    try:
        topic = get_topic(lsi,text)
        topict_list.append(topic)
    except:
        continue
print('nullptr 专栏文章主题分布：',Counter(topict_list))

nullptr 专栏文章主题分布： Counter({1: 13, 9: 10, 7: 9, 6: 5, 4: 4, 8: 3, 3: 1, 2: 1, 5: 1})
